In [1]:
import matplotlib.pyplot as plt # to make plots
import numpy as np # for array manipulation and math functions
import scipy.signal as scs # for signal analysis
import glob # for finding files
import os as os
import warnings
import array as arr
import uncertainties
from scipy import optimize
from scipy.optimize import curve_fit
from uncertainties import ufloat as uf
from uncertainties.umath import *
from uncertainties import unumpy as un
# import fit models from the lmfit package
from lmfit.models import ExponentialModel, GaussianModel, PolynomialModel

path2='../Versuch 526/Rohdaten/Tag 2/'
path1='../Versuch 526/Rohdaten/'
path_gaus='./gaus/'
path_raw='./raw/'
winkel=[]
winkeler=[]
for h in range(0,18):
    winkel.append(h*5+35)
    winkeler.append(1)
def E_kal(kanal,m,b):
    return m*kanal+b

def Effizienz(E,A,B,C,D):
    # return (P1+P2*np.log(E)+P4*np.log(E)**3+P5*np.log(E)**4    +P3*np.log(E)**2)/E
    return A-B*np.exp(C*(E+D))
def uEffizienz(E,A,B,C,D):
    # return (P1+P2*np.log(E)+P4*np.log(E)**3+P5*np.log(E)**4    +P3*np.log(E)**2)/E
    return A-B*exp(C*(E+D))
def Gausfit_1(winkel,x,y,namena,amplitude,center,sigma,dont_show):
    x1 = x
    y1 = y
    model = (GaussianModel(prefix='g1_'))
    params = model.make_params(g1_amplitude=amplitude,
                            g1_center=center,
                            g1_sigma=sigma)
    result = model.fit(y1, params, x=x1)
    # print(result.fit_report(min_correl=0.5))
    lolo1=[]
    lala1=[]
    for name,param in result.params.items():
        if result.errorbars==True:
            lala1.append(param.stderr)
        else:
            lala1.append(0)
        lolo1.append(param.value)
    comps = result.eval_components(x=x1)
    dely = result.eval_uncertainty(sigma=3)
    ergebnis = model.eval(params,x=x1)
    figure=plt.figure(figsize=(10,6))
    plt.title('Gaußkurvenfit an das korrigierte Cs-Streuspektrum bei '+str(winkel)+'$^{\circ}$',fontsize=15)
    plt.plot(x1,y1,"b-",linewidth=.2,label="Messwerte",)
    plt.plot(x1,result.best_fit,"r-",label="Gaußkurve",)
    plt.xlabel('Energie [keV]',fontsize=12)
    plt.ylabel('Zählrate',fontsize=12)
    plt.grid()
    plt.legend(fontsize=12)
    plt.savefig(namena+'.png')
    if (dont_show==1):
        plt.close(figure)
    #plt.show
    return lolo1,lala1

def klein_nishina(E,theta):
    t=theta*np.pi/180
    alpha=1/137
    m=511
    E_bar=E/(1+E/m*(1-np.cos(t)))
    return 0.5*(alpha*E_bar/(m*E))**2*(E_bar/E+E/E_bar-np.sin(t)**2)

def klein_nishina_ebar(E,theta,E_bar):
    t=theta*np.pi/180
    alpha=1/137
    m=511
    return 0.5*(alpha*E_bar/(m*E))**2*(E_bar/E+E/E_bar-np.sin(t)**2)

def klein_nishina_for_fit(theta,E_bar_off,theta_off,E_off):
    t=theta*np.pi/180
    alpha=1/137
    # E_bar_off=0
    # theta_off=0
    b=0
    E=646.78
    m=511
    E_bar=(E+E_off)/(1+(E+E_off)/m*(1-np.cos(t+theta_off*np.pi/180)))
    return 0.5*(alpha*(E_bar+E_bar_off)/(m*E))**2*((E_bar+E_bar_off)/E+E/(E_bar+E_bar_off)-np.sin(t+theta_off*np.pi/180)**2)+b

def klein_nishina_for_fit_no_E_off(theta,E_bar_off,theta_off):
    t=theta*np.pi/180
    alpha=1/137
    # E_bar_off=0
    E_off=0
    # theta_off=0
    b=0
    E=646.78
    m=511
    E_bar=(E+E_off)/(1+(E+E_off)/m*(1-np.cos(t+theta_off*np.pi/180)))
    return 0.5*(alpha*(E_bar+E_bar_off)/(m*E))**2*((E_bar+E_bar_off)/E+E/(E_bar+E_bar_off)-np.sin(t+theta_off*np.pi/180)**2)+b

def klein_nishina_for_fit_no_E_off_and_theta_off(theta,E_bar_off):
    t=theta*np.pi/180
    alpha=1/137
    # E_bar_off=0
    E_off=0
    theta_off=0
    b=0
    E=646.78
    m=511
    E_bar=(E+E_off)/(1+(E+E_off)/m*(1-np.cos(t+theta_off*np.pi/180)))
    return 0.5*(alpha*(E_bar+E_bar_off)/(m*E))**2*((E_bar+E_bar_off)/E+E/(E_bar+E_bar_off)-np.sin(t+theta_off*np.pi/180)**2)+b

def klein_nishina_for_fit_no_E_off_and_E_bar_off(theta,theta_off):
    t=theta*np.pi/180
    alpha=1/137
    E_bar_off=0
    E_off=0
    # theta_off=0
    b=0
    E=646.78
    m=511
    E_bar=(E+E_off)/(1+(E+E_off)/m*(1-np.cos(t+theta_off*np.pi/180)))
    return 0.5*(alpha*(E_bar+E_bar_off)/(m*E))**2*((E_bar+E_bar_off)/E+E/(E_bar+E_bar_off)-np.sin(t+theta_off*np.pi/180)**2)+b

def knff(var,b):
    alpha=((1/137)/511)**2
    return (alpha)*(var+b)

def kn_var(theta,E_bar):
    t=theta*np.pi/180
    E=646.78
    return 0.5*(E_bar/E)**2*(E_bar/E+E/E_bar-np.sin(t)**2)

def E(theta, E):
    t=theta*np.pi/180
    theta_off=0
    m=511
    return E/(1+E/m*(1-np.cos(t-theta_off*np.pi/180)))

def Gausfit_11(x,y,amplitude,center,sigma):
    x1 = x
    y1 = y
    model = (GaussianModel(prefix='g1_'))
    params = model.make_params(g1_amplitude=amplitude,
                            g1_center=center,
                            g1_sigma=sigma)
    result = model.fit(y1, params, x=x1)
    #print(result.fit_report(min_correl=0.5))
    lolo1=[]
    lala1=[]
    for name,param in result.params.items():
        if result.errorbars==True:
            lala1.append(param.stderr)
        else:
            lala1.append(0)
        lolo1.append(param.value)
    comps = result.eval_components(x=x1)
    dely = result.eval_uncertainty(sigma=3)
    ergebnis = model.eval(params,x=x1)
    return lolo1,lala1,result.best_fit

def Gausfit_2(x,y,amplitude,center,sigma):
    x1 = x
    y1 = y
    model = (GaussianModel(prefix='g1_') +
             GaussianModel(prefix='g2_'))
    params = model.make_params(g1_amplitude=amplitude,
                            g1_center=center,
                            g1_sigma=sigma,
                            g2_amplitude=amplitude,
                            g2_center=center,
                            g2_sigma=sigma)
    result = model.fit(y1, params, x=x1)
    print(result.fit_report(min_correl=0.5))
    lolo1=[]
    lala1=[]
    for name,param in result.params.items():
        if result.errorbars==True:
            lala1.append(param.stderr)
        else:
            lala1.append(0)
        lolo1.append(param.value)
    comps = result.eval_components(x=x1)
    dely = result.eval_uncertainty(sigma=3)
    ergebnis = model.eval(params,x=x1)
    return lolo1,lala1,result.best_fit

def Plot_raw(K,Kbg,I,Ibg,name,path,dont_show_and_save):
    figure=plt.figure(figsize=(10,6))
    plt.title('Rohdaten Cs-Streuung an 1mm Al-Target bei '+ str(name)+'$^{\circ}$',fontsize=15)
    plt.plot(K,I,"b", label='mit Cs')
    plt.plot(Kbg,Ibg,'r',label='Hintergrund')
    plt.xlabel('Energie [keV]',fontsize=12)
    plt.ylabel('Zählrate',fontsize=12)
    plt.grid()
    plt.legend()
    if dont_show_and_save==1:
        plt.savefig(path+str(name)+'.png')
        plt.close(figure)
    return 0

def Gauskurvenauswertung(winkel,ext,bg_ext,path,raw_path,gaus_path,show,amplitude,center,sigma):
    data=np.loadtxt(path+str(winkel)+ext)
    data_bg=np.loadtxt(path+str(winkel)+bg_ext)
    kanal=E_kal(data[:,0],0.16253798, -11.78733577)
    kanal_bg=E_kal(data_bg[:,0],0.16253798, -11.78733577)
    zahl=data[:,1]
    zahl_bg=data_bg[:,1]
    Plot_raw(kanal,kanal_bg,zahl,zahl_bg,str(winkel),raw_path,1)
    par,stderr=Gausfit_1(winkel,kanal,zahl-zahl_bg,gaus_path+str(winkel)+'gaus',amplitude,center,sigma,show)
    return par,stderr

def Gradenfit(x,y):
    anpassungs_werte, co_var = curve_fit(E_kal,x,y)
    perr= np.sqrt(np.diag(co_var))
    print("Durch die Anpassung bestimmte Parameter:", anpassungs_werte, "+/-", perr, "nm")
    #Rfit    = funktion(U2[h], *anpassungs_werte)
    Rfit=[]
    for E in x:
        Rfit.append(E_kal(E, *anpassungs_werte))
    return Rfit

def Fit(x,y,s,funktion,d_y):
    anpassungs_werte, co_var = curve_fit(funktion,x,y,sigma=d_y,absolute_sigma=True,p0=s)
    perr= np.sqrt(np.diag(co_var))
    print("Durch die Anpassung bestimmte Parameter:", anpassungs_werte, "+/-", perr, "nm")
    #Rfit    = funktion(U2[h], *anpassungs_werte)
    Rfit=[]
    for E in x:
        Rfit.append(funktion(E, *anpassungs_werte))
    return Rfit

def Fit2(x,y,s,funktion):
    anpassungs_werte, co_var = curve_fit(funktion,x,y,p0=s)
    perr= np.sqrt(np.diag(co_var))
    print("Durch die Anpassung bestimmte Parameter:", anpassungs_werte, "+/-", perr, "nm")
    #Rfit    = funktion(U2[h], *anpassungs_werte)
    Rfit=[]
    for E in x:
        Rfit.append(funktion(E, *anpassungs_werte))
    return Rfit

def Fit_smooth(x,y,x_smooth,s,funktion,d_y):
    anpassungs_werte, co_var = curve_fit(funktion,x,y,sigma=d_y,absolute_sigma=True,p0=s)
    perr= np.sqrt(np.diag(co_var))
    print("Durch die Anpassung bestimmte Parameter:", anpassungs_werte, "+/-", perr, "nm")
    #Rfit    = funktion(U2[h], *anpassungs_werte)
    Rfit=[]
    for E in x_smooth:
        Rfit.append(funktion(E, *anpassungs_werte))
    return Rfit

def pol_cords(r,theta):
    t=theta*np.pi/180
    return r*np.sin(t),r*np.cos(t)

def pol_cords_for_list_x(r,theta):
    x=[]
    i=0
    for t in theta:
        x.append(r[i]*np.sin(t*np.pi/180))
        i=i+1
    return x
def pol_cords_for_list_y(r,theta):
    y=[]
    i=0
    for t in theta:
        y.append(r[i]*np.cos(t*np.pi/180))
        i=i+1
    return y

def upol_cords_for_list_x(r,theta):
    x=[]
    i=0
    # for t in theta:
    x.append(r*sin(theta*np.pi/180))
    i=i+1
    return x
def upol_cords_for_list_y(r,theta):
    y=[]
    i=0
    # for t in theta:
    y.append(r*cos(theta*np.pi/180))
    i=i+1
    return y

<>:56: SyntaxWarning: invalid escape sequence '\c'
<>:194: SyntaxWarning: invalid escape sequence '\c'
<>:56: SyntaxWarning: invalid escape sequence '\c'
<>:194: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Banton\AppData\Local\Temp\ipykernel_1056\3513899254.py:56: SyntaxWarning: invalid escape sequence '\c'
  plt.title('Gaußkurvenfit an das korrigierte Cs-Streuspektrum bei '+str(winkel)+'$^{\circ}$',fontsize=15)
C:\Users\Banton\AppData\Local\Temp\ipykernel_1056\3513899254.py:194: SyntaxWarning: invalid escape sequence '\c'
  plt.title('Rohdaten Cs-Streuung an 1mm Al-Target bei '+ str(name)+'$^{\circ}$',fontsize=15)


In [ ]:
i=0
A=1.07229309e+00
B=3.26103569e-02 
C=2.34665088e-03 
D=2.16681106e+02
Amplitudes_cor=[]
Amplitudes_corer=[]
centers=[]
centerser=[]
if 'params' not in locals():
    params=[]
    params_stderr=[]
    amp=[]
    step=(E_kal(3500,0.16253798, -11.78733577)-E_kal(1200,0.16253798, -11.78733577))/17
    for w in winkel:
        amp.append(E_kal(3500,0.16253798, -11.78733577)-step*i)
        a1,a2=Gauskurvenauswertung(w,'.txt','b.txt',path2,path_raw,path_gaus,1,2000,amp[i],35)
        params.append(a1)
        params_stderr.append(a2)
        i=i+1
    print(params)
vamp=un.uarray(params,params_stderr)
for para in params:
    centers.append(para[1])
for vamp in vamp:
    Amplitudes_cor.append((vamp[0]/uEffizienz(vamp[1],A,B,C,D)).n)
    Amplitudes_corer.append((vamp[0]/uEffizienz(vamp[1],A,B,C,D)).s)
for para in params_stderr:
    centerser.append(para[1])

S=[660]
x_smooth=np.linspace(35,120,num=200)
fit=Fit_smooth(winkel,centers,x_smooth,S,E,centerser)
figure=plt.figure(figsize=(12,6))
plt.title('Kurvenfit an Energie-Streuwinkel-Relation',fontsize=15)
plt.plot(centers,winkel,"b .",linewidth=1)
plt.plot(fit,x_smooth,"r-",linewidth=1)
plt.errorbar(centers,winkel,winkeler,centerser,'k.',capsize=3,elinewidth=1,markersize=0,label='Fehlerbalken')
plt.xlabel('Energie [keV]',fontsize=12)
plt.ylabel('Streuwinkel '+ '$\Theta$ [$^{\circ}$]',fontsize=12)
plt.grid()


x=[]
y=[]
kn=[]
x_kn=[]
y_kn=[]
x_fp=[]
y_fp=[]
x_fit=[]
y_fit=[]
kn_fp=[]
norm_ampl=[]
norm_ampler=[]
kn_vars=[]
for i in range(0,18):
    kn_vars.append(kn_var(winkel[i],centers[i]))
normierung=klein_nishina(646.78086302,winkel[8])/Amplitudes_cor[8]
# normierung=klein_nishina_ebar(646.78086302,winkel[8],centers[8])/Amplitudes_cor[8]
for amp in Amplitudes_cor:
    norm_ampl.append(normierung*amp)
for amp in Amplitudes_corer:
    norm_ampler.append(normierung*amp)
dict1=[list() for x in range(2)]
dict2=[list() for x in range(2)]
dict1[0]=norm_ampl
dict1[1]=winkel
dict2[0]=norm_ampler
dict2[1]=winkeler
# normuamp=np.concentrate((norm_ampl,winkel))
# normuamper=np.concatenate(norm_ampler,winkeler)
unorm=un.uarray(dict1,dict2)
# print(unorm)
S=[0,0,0]
fit=Fit2(winkel,norm_ampl,S,klein_nishina_for_fit)
fit2=Fit2(winkel,norm_ampl,[0,0],klein_nishina_for_fit_no_E_off)
fit3=Fit2(winkel,norm_ampl,[0],klein_nishina_for_fit_no_E_off_and_theta_off)
fit4=Fit2(winkel,norm_ampl,[0],klein_nishina_for_fit_no_E_off_and_E_bar_off)